## Model training

In [1]:
from transformers import TrainingArguments, Trainer
import pickle
from transformers import (
    AutoModelForSequenceClassification,
    AutoModelForSeq2SeqLM,
    AutoConfig,
    BertModel,
)
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import torch.nn as nn
from transformers.modeling_outputs import SequenceClassifierOutput

/home/cc/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-26 03:13:09.437773: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Loading the data


In [2]:
with open('train_dataset_tokenized.pkl', 'rb') as file:
    train_dataset = pickle.load(file)

with open('val_data_tokenized.pkl', 'rb') as file:
    val_dataset = pickle.load(file)

with open('test_data_tokenized.pkl', 'rb') as file:
    test_dataset = pickle.load(file)

with open('train_dataset_full_tokenized.pkl', 'rb') as file:
    train_dataset_full = [pickle.load(file)]

with open('augmented_train_dataset_tokenized.pkl', 'rb') as file:
    train_dataset_augmented = pickle.load(file)

### Setting up the training arguments

In [3]:
args = TrainingArguments(
        output_dir="./output",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=8 ,
        per_device_eval_batch_size=8 ,
        num_train_epochs=3,
        warmup_ratio= 0.1,
        weight_decay= 0.001,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        save_total_limit=1,
            )

pre_trained_BERTmodel='bert-large-uncased'
BERT_tokenizer=AutoTokenizer.from_pretrained(pre_trained_BERTmodel)

### Modifying Bert for our classification Task

In [4]:
class BertModelWithCustomLossFunction(nn.Module):
    def __init__(self):
        super(BertModelWithCustomLossFunction, self).__init__()
        self.num_labels = 64
        self.bert = BertModel.from_pretrained(
            pre_trained_BERTmodel, num_labels=self.num_labels
        )
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(1024, self.num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )

        output = self.dropout(outputs.pooler_output)
        logits = self.classifier(output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

### Setting up metrics for accuracy, precision, recall and f1

In [5]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

### Training the model

In [6]:
import warnings
warnings.filterwarnings("ignore")


In [7]:
for train in [train_dataset_full, train_dataset, train_dataset_augmented]:
  best_accuracy = 0
  for train_data in train:
    BERT_model = BertModelWithCustomLossFunction()
    trainer = Trainer(
          model = BERT_model,
          args = args,
          train_dataset=train_data,
          eval_dataset=val_dataset,
          tokenizer=BERT_tokenizer,
          compute_metrics=compute_metrics,)
    trainer.train()
    evaluation_metrics = trainer.predict(test_dataset)
    accuracy = evaluation_metrics.metrics['test_accuracy']
    best_accuracy = max(accuracy, best_accuracy)
    torch.cuda.empty_cache()
  print(f"Best Test Accuracy for this training dataset: {accuracy}")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
libibverbs: Warning: couldn't open config directory '/etc/libibverbs.d'.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.794267,0.841893,0.828257,0.841893,0.823365
2,1.736900,0.383715,0.913394,0.914087,0.913394,0.909503
3,1.736900,0.344701,0.917422,0.920600,0.917422,0.916044


Best Test Accuracy for this training dataset: 0.9125581395348837


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,4.076813,0.029204,0.006747,0.029204,0.009313
2,No log,3.950717,0.069486,0.060676,0.069486,0.042101
3,No log,3.847492,0.133938,0.132056,0.133938,0.099718


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,4.151068,0.029204,0.014804,0.029204,0.016411
2,No log,4.005015,0.034240,0.040095,0.034240,0.021546
3,No log,3.932772,0.088620,0.102344,0.088620,0.065755


...

Best Test Accuracy for this training dataset: 0.10511627906976745


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,4.014378,0.045317,0.016556,0.045317,0.016300
2,No log,3.717148,0.165156,0.224983,0.165156,0.126444
3,No log,3.563570,0.239678,0.287328,0.239678,0.190705


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,4.159652,0.024169,0.019703,0.024169,0.011212
2,No log,4.070272,0.047331,0.056689,0.047331,0.023524
3,No log,3.987136,0.064451,0.084944,0.064451,0.049641


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,4.070035,0.038268,0.039100,0.038268,0.022016
2,No log,3.820243,0.139980,0.139199,0.139980,0.096027
3,No log,3.688543,0.207452,0.228843,0.207452,0.160914


Best Test Accuracy for this training dataset: 0.17767441860465116
